In [ ]:
%matplotlib inline

import matplotlib.pyplot as plt

import numpy as np
import librosa
import librosa.display
import soundfile as sf

from IPython.display import Audio

## TODO:

Maybe pick peaks, or lowpass filter or something. I guess we are taking all the indices surrounding a peak in this method.

Especially for the FFT case, try to fin most important bands somehow.

In [ ]:
DEO_PATH = "/home/kureta/Music/deo.wav"
SAMPLE_RATE = 48000
NFFT = 1024
HOP_LENGTH = 512

In [ ]:
audio, sr = librosa.load(DEO_PATH, sr=SAMPLE_RATE, mono=False)

## Canon for 1 Voice in STFT

In [ ]:
stft = librosa.stft(audio, n_fft=NFFT, hop_length=HOP_LENGTH)
sdb = librosa.amplitude_to_db(np.abs(stft), top_db=120)
mono = sdb.mean(axis=0)

In [ ]:
mono.shape

In [ ]:
indices = np.where(mono.mean(axis=1) > -24.)[0]
indices = sorted(indices, key=lambda x: mono.mean(axis=1)[x])
indices = np.array(indices)
indices = indices[(indices > 4) & (indices < 128)]
print(len(indices))

In [ ]:
for i in range(27):
    print(i*7.5-97.5)

In [ ]:
sounds = []
for idx in indices:
    pick = np.zeros_like(stft)
    pick[:, idx, :] = 1
    sounds.append(librosa.istft(stft * pick, hop_length=HOP_LENGTH))

In [ ]:
Audio(data=np.concatenate(np.cumsum(sounds, axis=0), axis=1), rate=SAMPLE_RATE)

In [ ]:
Audio(data=np.concatenate(sounds, axis=1), rate=SAMPLE_RATE)

In [ ]:
sf.write('/home/kureta/Music/deo-solo.wav', np.concatenate(sounds, axis=1).T, SAMPLE_RATE, subtype='PCM_24')

In [ ]:
for idx, sound in enumerate(sounds):
    sf.write(f'/home/kureta/Music/deo-{idx+1:02d}.wav', sound.T, SAMPLE_RATE, subtype='PCM_24')

## Canon for 1 Voice in FFT

In [ ]:
fft = np.fft.rfft(audio)
sdb = librosa.amplitude_to_db(np.abs(fft), top_db=120)
mono = sdb.mean(axis=0)
mono[8000:] = -120

In [ ]:
peaks = librosa.util.peak_pick(mono[:8000], pre_max=100, post_max=100, pre_avg=100, post_avg=100, delta=10, wait=50)
plt.vlines(peaks, 0, 1)
print(len(peaks))

In [ ]:
plt.plot(mono[:8000])

In [ ]:
# indices = np.where(mono > 40.)[0]
indices = peaks
print(len(indices))
np.random.shuffle(indices)

In [ ]:
sounds = []
for idx in indices:
    pick = np.zeros_like(fft)
    pick[:, idx] = 1
    sounds.append(np.fft.irfft(fft * pick))

In [ ]:
Audio(data=np.concatenate(np.cumsum(sounds, axis=0), axis=1), rate=SAMPLE_RATE)

In [ ]:
Audio(data=np.concatenate(sounds, axis=1), rate=SAMPLE_RATE)